In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt


## data preprocessing

In [2]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    r"C:\Users\Ashish yadav\100DL\12.Brain tumour detection system\Training",
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

test_generator = train_datagen.flow_from_directory(
    r"C:\Users\Ashish yadav\100DL\12.Brain tumour detection system\Testing",
    target_size=(224,224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 4571 images belonging to 4 classes.
Found 262 images belonging to 4 classes.


## BUild model by transfer learning

In [3]:
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')
])


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 8s 0us/step


## compile model

In [4]:
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


## training model

In [6]:
history = model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=10
)


Epoch 1/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 121s 817ms/step - accuracy: 0.4362 - loss: 1.1960 - val_accuracy: 0.3702 - val_loss: 1.3594
Epoch 2/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 114s 794ms/step - accuracy: 0.5071 - loss: 1.0630 - val_accuracy: 0.5038 - val_loss: 1.3921
Epoch 3/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 115s 805ms/step - accuracy: 0.5384 - loss: 1.0185 - val_accuracy: 0.4580 - val_loss: 1.3765
Epoch 4/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 121s 843ms/step - accuracy: 0.5668 - loss: 0.9819 - val_accuracy: 0.4771 - val_loss: 1.3357
Epoch 5/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 113s 791ms/step - accuracy: 0.5846 - loss: 0.9575 - val_accuracy: 0.4618 - val_loss: 1.3398
Epoch 6/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 114s 793ms/step - accuracy: 0.5996 - loss: 0.9185 - val_accuracy: 0.5191 - val_loss: 1.2899
Epoch 7/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 116s 806ms/step - accuracy: 0.6093 - loss: 0.9093 - val_accuracy: 0.4542 - val_loss: 1.3425
Epoch 8/10
143/143 ━━━━━━━━━━━━━━━━━━━━ 113s 787ms/step - accuracy: 0.6080 -

## evaluate model

In [8]:
model.evaluate(test_generator)


9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 623ms/step - accuracy: 0.5191 - loss: 1.3084


[1.308351993560791, 0.5190839767456055]

## improve accuracy

In [10]:
## after initia training ----fine tuning for better accuraccy

base_model.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(train_generator, validation_data=test_generator, epochs=5)


Epoch 1/5
143/143 ━━━━━━━━━━━━━━━━━━━━ 478s 3s/step - accuracy: 0.4373 - loss: 19.7347 - val_accuracy: 0.3092 - val_loss: 52.0876
Epoch 2/5
143/143 ━━━━━━━━━━━━━━━━━━━━ 457s 3s/step - accuracy: 0.6633 - loss: 1.2054 - val_accuracy: 0.3092 - val_loss: 36.7635
Epoch 3/5
143/143 ━━━━━━━━━━━━━━━━━━━━ 451s 3s/step - accuracy: 0.7572 - loss: 0.7512 - val_accuracy: 0.3092 - val_loss: 27.5802
Epoch 4/5
143/143 ━━━━━━━━━━━━━━━━━━━━ 451s 3s/step - accuracy: 0.8095 - loss: 0.5576 - val_accuracy: 0.3092 - val_loss: 19.0508
Epoch 5/5
143/143 ━━━━━━━━━━━━━━━━━━━━ 483s 3s/step - accuracy: 0.8464 - loss: 0.4161 - val_accuracy: 0.3092 - val_loss: 17.0075


## saved model

In [11]:
model.save("brain_tumor_model.h5")
